# Class 4 - Tree-based models

**Packages import**

In [ ]:
# %%capture
# !pip install scipy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier as CART
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score

pd.options.mode.chained_assignment = None

## Dataset preprocessing

We'll use IMDB 5000 Movies dataset in the analysis

In [ ]:
dataset = pd.read_csv("IMDB.csv")
dataset.head()

**Data profiling**

Comprehensive data reports for Pandas DataFrames may be generated using [`ydata-profiling`](https://github.com/ydataai/ydata-profiling) package - **IMDB_Report.zip** contains profiling report for IMDB dataset.
```
profile = ProfileReport(dataset, title="IMDB Movies Profiling Report")
profile.to_file(output_file="IMDB_Report.html")
```

**Initial EDA (Exploratory Data Analysis)**

In [ ]:
# Checking categorical columns
dataset.describe(include=["O"])

In [ ]:
# Dropping columns with high cardinality and imbalanced classes
dataset.drop(
    [
        "color",
        "director_name",
        "actor_2_name",
        "actor_1_name",
        "movie_title",
        "actor_3_name",
        "plot_keywords",
        "movie_imdb_link",
        "language",
        "country",
        "content_rating",
    ],
    axis=1,
    inplace=True,
)

In [ ]:
# Drop duplicates
print(dataset.shape)
dataset.drop_duplicates(inplace=True)
print(dataset.shape)

In [ ]:
# Check null values
dataset.isnull().sum()

In [ ]:
# Dropping missing values
dataset.dropna(inplace=True)
dataset.shape

In [ ]:
numeric_dataset = dataset.select_dtypes(np.number)
f, axes = plt.subplots(4, 4, figsize=[15, 15])
plt.tight_layout(pad=0.4, w_pad=1.0, h_pad=1.0)
for n, col in enumerate(numeric_dataset):
    sns.regplot(x=col, y="imdb_score", data=dataset, ax=axes[n // 4, n % 4])

**Detecting outliers**

A raw score x is converted into a standard score by

$$ z= \frac{x-\mu}{\sigma}  $$

where:

* μ is the mean of the population,
* σ is the standard deviation of the population.

In [ ]:
stats.zscore(numeric_dataset)

In [ ]:
# Removing outliers
dataset = dataset[(np.abs(stats.zscore(numeric_dataset)) < 9).all(axis=1)]
dataset.shape

**Feature engineering**

In [ ]:
# Splitting genres column values
dataset["genres"] = dataset.genres.str.split("|")
dataset["genres"]

In [ ]:
# Getting distinct categories
categories = set(dataset.genres.explode())
categories

In [ ]:
# Encode each movie's classification
for cat in categories:
    dataset[cat] = dataset.genres.apply(lambda s: int(cat in s))
dataset.head()

In [ ]:
# Drop genres column
dataset.drop("genres", axis=1, inplace=True)

**EDA on cleaned data**

In [ ]:
f, axes = plt.subplots(4, 4, figsize=[15, 15])
plt.tight_layout(pad=0.4, w_pad=1.0, h_pad=1.0)
for n, col in enumerate(dataset.columns[0:16]):
    sns.regplot(x=col, y="imdb_score", data=dataset, ax=axes[n // 4, n % 4])

In [ ]:
f, axes = plt.subplots(6, 4, figsize=[15, 15])
plt.tight_layout(pad=0.4, w_pad=1.0, h_pad=1.0)
for n, col in enumerate(dataset.columns[16:]):
    sns.barplot(x=col, y="imdb_score", data=dataset, ax=axes[n // 4, n % 4]).set_ylim(
        5,
    )
f.delaxes(axes[5, 2])
f.delaxes(axes[5, 3])

**Switching from regression to binary classification task**

In [ ]:
print(np.median(dataset.imdb_score))
dataset["imdb_score"] = np.where(dataset["imdb_score"] >= 6.0, 1, 0)
dataset["imdb_score"].value_counts()

**Splitting data into train and test subsets**

In [ ]:
X = dataset.drop("imdb_score", axis=1)
y = dataset["imdb_score"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

**Decision Trees**

From: [sklearn docs](https://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart)

**ID3** (Iterative Dichotomiser 3) was developed in 1986 by Ross Quinlan. The algorithm creates a multiway tree, finding for each node (i.e. in a greedy manner) the categorical feature that will yield the largest information gain for categorical targets. Trees are grown to their maximum size and then a pruning step is usually applied to improve the ability of the tree to generalise to unseen data.

**C4.5** is the successor to ID3 and removed the restriction that features must be categorical by dynamically defining a discrete attribute (based on numerical variables) that partitions the continuous attribute value into a discrete set of intervals. C4.5 converts the trained trees (i.e. the output of the ID3 algorithm) into sets of if-then rules. These accuracy of each rule is then evaluated to determine the order in which they should be applied. Pruning is done by removing a rule’s precondition if the accuracy of the rule improves without it.

**C5.0** is Quinlan’s latest version release under a proprietary license. It uses less memory and builds smaller rulesets than C4.5 while being more accurate.

**CART** (Classification and Regression Trees) is very similar to C4.5, but it differs in that it supports numerical target variables (regression) and does not compute rule sets. CART constructs binary trees using the feature and threshold that yield the largest information gain at each node.

**scikit-learn uses an optimised version of the CART algorithm**

CART algorithm pick variables and cutoff threshold using:
 1. __for classification__: minimization of node's heterogeneity (Gini index or entropy) 
 2. __for regression__: minimizing error of prediction (e.g. sum of squares of residuals)

In [ ]:
X_train_CART, X_val_CART, y_train_CART, y_val_CART = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

In [ ]:
imdb_tree = CART(random_state=42, ccp_alpha=0.0).fit(X_train_CART, y_train_CART)

[**Pruning CART tree (cost based)**](https://scikit-learn.org/stable/modules/tree.html#minimal-cost-complexity-pruning)


In [ ]:
path = imdb_tree.cost_complexity_pruning_path(X_train_CART, y_train_CART)
ccp_alphas, impurities = path.ccp_alphas[::4], path.impurities[::4]

In [ ]:
imdb_trees = []
for cp in ccp_alphas:
    imdb_trees.append(
        CART(random_state=42, ccp_alpha=cp).fit(X_train_CART, y_train_CART)
    )

In [ ]:
for order, ind in [("First", 0), ("Last", -1)]:
    print(
        f"{order} tree with ccp_alpha: {ccp_alphas[ind]:.5f}, "
        + f"nodes: {imdb_trees[ind].tree_.node_count}, leaves: {imdb_trees[ind].get_n_leaves()}"
    )

In [ ]:
val_scores = [accuracy_score(y_val_CART, tree.predict(X_val_CART)) for tree in imdb_trees]
train_scores = [accuracy_score(y_train_CART, tree.predict(X_train_CART)) for tree in imdb_trees]

plt.figure(figsize=(10, 6))
plt.plot(ccp_alphas, train_scores, label="train", drawstyle="steps-post")
plt.plot(ccp_alphas, val_scores, label="validation", drawstyle="steps-post")
plt.xlabel(r"$\alpha$")
plt.ylabel("Accuracy")
plt.title("Accuracy vs complexity parameter for training and validation sets")
plt.legend()
plt.show()

In [ ]:
# Complexity (cost) that produce the best tree
Best_CART = imdb_trees[np.argmax(val_scores)]
Best_CART.ccp_alpha

In [ ]:
# Visualize the tree
plt.figure(figsize=(15, 10))
_ = plot_tree(Best_CART, feature_names=X_train.columns, filled=True)  # imdb_trees[-1],

In [ ]:
# Accuracy of the best tree
round(max(val_scores) * 100, 3)

### Ensemble tree-based methods

[Ensemble learning](https://scikit-learn.org/stable/modules/ensemble.html) helps improve final model performance by combining results of underlying models (e.g. random forest is combination of decision trees).

Two families of ensemble methods are usually distinguished:

- In **averaging methods**, the main principle is to build several estimators **independently** and then to average their predictions. On average, the combined estimator is usually better than any of the single base estimator.

> Example: Random forests

- By contrast, in **boosting methods**, base estimators are built **sequentially** and the following models tries to reduce the error of the combined estimator.

> Example: Boosted trees

<img src="https://hpccsystems.com/wp-content/uploads/2022/09/LearningTrees-1.png" width=500>

[Source](https://hpccsystems.com/resources/learning-trees-a-guide-to-decision-tree-based-machine-learning/)

[**Random forests**](https://scikit-learn.org/stable/modules/ensemble.html#random-forests)

Random forest is a collection of 'weak' decision trees providing good performance together.

Trees are weakned using multiple techniques:
* bootstrap sample, potentially on subset of available data
* limiting number of features
* no pruning

In [ ]:
# Tuning number of trees and number of features
grid = {
    "n_estimators": [50, 100, 200, 300, 400],
    "max_features": np.linspace(1, X_train.shape[1], 5).astype(int),
}
tuning_res_rf = GridSearchCV(
    RFC(random_state=42), param_grid=grid, scoring="accuracy", n_jobs=1, cv=3, verbose=2
)
tuning_res_rf.fit(X_train, y_train)

In [ ]:
n_trees = grid["n_estimators"]
max_features = grid["max_features"]
arr = tuning_res_rf.cv_results_["mean_test_score"].reshape(
    len(max_features), len(n_trees)
)
df = pd.DataFrame(arr, columns=n_trees, index=max_features)

In [ ]:
p = sns.heatmap(df, annot=True, fmt=".3f", cmap="viridis")
p.set_xlabel("n_estimators")
p.set_ylabel("# features");

In [ ]:
print(tuning_res_rf.best_params_)
Best_RF = tuning_res_rf.best_estimator_

In [ ]:
# Plot the feature importances of the forest
importances = Best_RF.feature_importances_
std = np.std([tree.feature_importances_ for tree in Best_RF.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

num_feat = 6
plt.figure(figsize=[15, 8])
plt.title("Feature importances", fontsize=20)
plt.bar(
    range(num_feat)[:num_feat],
    importances[indices][:num_feat],
    color="g",
    yerr=std[indices][:num_feat],
    align="center",
)
plt.xticks(range(num_feat)[:num_feat], X_train.columns[indices[:num_feat]])
plt.xlim([-1, num_feat])
plt.ylabel("Impurity reduction", fontsize=15);

[**Gradient Boosted Trees**](https://scikit-learn.org/stable/modules/ensemble.html#gradient-tree-boosting)

In [ ]:
# Tuning number of iterations and percentage of bootstrap sample
dist = {"n_estimators": stats.randint(100, 400), "subsample": stats.uniform()}
tuning_res_gbc = RandomizedSearchCV(
    GBC(random_state=42),
    param_distributions=dist,
    scoring="accuracy",
    n_iter=25,
    n_jobs=1,
    cv=3,
    verbose=2,
)
tuning_res_gbc.fit(X_train, y_train)

In [ ]:
print(tuning_res_gbc.best_params_)
Best_GBT = tuning_res_gbc.best_estimator_

In [ ]:
# Plot feature importance
feature_importance = Best_GBT.feature_importances_
# Make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + 0.5
num_feat = 6

plt.figure(figsize=[12, 8])
plt.barh(
    pos[-num_feat:],
    feature_importance[sorted_idx][-num_feat:],
    align="center",
    alpha=0.75,
)
plt.yticks(pos[-num_feat:], X_train.columns[sorted_idx][-num_feat:])
plt.xlabel("Relative Importance")
plt.title("Variable Importance");

**Comparing results of Decision Tree, Random Forest and Gradient Boosted Trees**

In [ ]:
models = [Best_CART, Best_RF, Best_GBT]
accuracies_test = [accuracy_score(y_test, m.predict(X_test)) for m in models]

In [ ]:
fig = plt.bar(
    ["CART", "Random Forest", "Gradient Boosted Trees"],
    accuracies_test,
    color=["red", "green", "blue"],
    alpha=0.75
)
plt.bar_label(fig, fmt="%.3f")
plt.ylabel("Accuracy");

### Exercises

Tune Gradient Boosted Trees using `GridSearchCV` setup as for Random Forest (with tuning `n_estimators` and `max_features`). Extract best estimator form the results of grid search.

Calculate accuracy for produced Boosted Trees on test data. Add the result as 4th bar on the previously produced barplot.